In [1]:
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds


2023-01-10 14:26:48.604057: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import BatchNormalization

In [3]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()        

In [4]:
CK_PATH="CK_PLUS/CK+48" 
CK_PATH= pathlib.Path(CK_PATH)
image_count = len(list(CK_PATH.glob('*/*.png')))
#print(image_count)

981


In [5]:
anger = list(CK_PATH.glob('anger/*'))
contempt = list(CK_PATH.glob('contempt/*'))
disgust = list(CK_PATH.glob('disgust/*'))
fear = list(CK_PATH.glob('fear/*'))
happy = list(CK_PATH.glob('happy/*'))
sadness = list(CK_PATH.glob('sadness/*'))
surprise = list(CK_PATH.glob('surprise/*'))
#PIL.Image.open(str(happy[0]))
#print image

In [65]:
batch_size=32
img_height = 48
img_width = 48

In [68]:
train_ds = tf.keras.utils.image_dataset_from_directory(
CK_PATH,
validation_split=0.15,
subset="training",
seed=123,
image_size=(img_height, img_width),
batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
CK_PATH,
validation_split=0.15,
subset="validation",
seed=123,
image_size=(img_height, img_width),
batch_size=batch_size)

Found 981 files belonging to 7 classes.
Using 834 files for training.
Found 981 files belonging to 7 classes.
Using 147 files for validation.


In [71]:
class_names = train_ds.class_names
class_no = len(class_names)
print(class_no )

7


In [ ]:
#--------Data Augmentation------------



#-------------------------------------

In [55]:
"""
plt.figure(figsize=(10, 10)) #first 9 image with class
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")
        
"""

'\nplt.figure(figsize=(10, 10)) #first 9 image with class\nfor images, labels in train_ds.take(1):\n    for i in range(9):\n        ax = plt.subplot(3, 3, i + 1)\n        plt.imshow(images[i].numpy().astype("uint8"))\n        plt.title(class_names[labels[i]])\n        plt.axis("off")\n        \n'

In [56]:
for image_batch, labels_batch in train_ds:
    input_shape = image_batch.shape
    label_shape = labels_batch.shape
    print(input_shape)
    #print(label_shape )
    break 

(32, 48, 48, 3)


In [62]:
image_batch = image_batch.reshape(input_shape[0], input_shape[1], input_shape[2], 3)
image_batch = image_batch.astype('float32')
image_batch = image_batch/ 255.

In [72]:
#FIRST PROPOSED MODEL
#-------------------
model = keras.Sequential()
#model.add(tf.keras.layers.Rescaling(1./255))

#--convolutional layer x3---
model.add(tf.keras.layers.Conv2D(6, kernel_size=(3, 3), activation='relu', input_shape=(image_batch.shape[1], image_batch.shape[2], 3) ))#image_batch.shape
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=(image_batch.shape[1], image_batch.shape[2], 3) ))#image_batch.shape
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(image_batch.shape[1], image_batch.shape[2], 3) ))#image_batch.shape
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

#--Fully Connected Layers x2---

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(class_no, activation='softmax'))


#-------------------


In [73]:
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_39 (Conv2D)          (None, 46, 46, 6)         168       
                                                                 
 batch_normalization_47 (Bat  (None, 46, 46, 6)        24        
 chNormalization)                                                
                                                                 
 max_pooling2d_39 (MaxPoolin  (None, 23, 23, 6)        0         
 g2D)                                                            
                                                                 
 conv2d_40 (Conv2D)          (None, 21, 21, 16)        880       
                                                                 
 batch_normalization_48 (Bat  (None, 21, 21, 16)       64        
 chNormalization)                                                
                                                     

In [74]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [76]:
model.fit(train_ds, epochs=10,verbose=1,validation_data=test_ds)
#validation_split=0.15


Epoch 1/10
27/27 [==============================] - 1s 38ms/step - loss: 0.1685 - accuracy: 0.9496 - val_loss: 0.3461 - val_accuracy: 0.9456
Epoch 2/10
27/27 [==============================] - 1s 29ms/step - loss: 0.1876 - accuracy: 0.9424 - val_loss: 0.3301 - val_accuracy: 0.9456
Epoch 3/10
27/27 [==============================] - 1s 29ms/step - loss: 0.1976 - accuracy: 0.9329 - val_loss: 0.2526 - val_accuracy: 0.9456
Epoch 4/10
27/27 [==============================] - 1s 29ms/step - loss: 0.1386 - accuracy: 0.9532 - val_loss: 0.1943 - val_accuracy: 0.9660
Epoch 5/10
27/27 [==============================] - 1s 29ms/step - loss: 0.1437 - accuracy: 0.9508 - val_loss: 0.1997 - val_accuracy: 0.9592
Epoch 6/10
27/27 [==============================] - 1s 30ms/step - loss: 0.1695 - accuracy: 0.9472 - val_loss: 0.1907 - val_accuracy: 0.9388
Epoch 7/10
27/27 [==============================] - 1s 29ms/step - loss: 0.1172 - accuracy: 0.9640 - val_loss: 0.1454 - val_accuracy: 0.9592
Epoch 8/10
27

In [41]:




# --------DRAFT--------

#normalization_layer = tf.keras.layers.Rescaling(1./255) # Normalize pixel values to be between 0 and 1

#normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
#image_batch, labels_batch = next(iter(normalized_ds))


#def relu(x): #ReLU Activation Function
 #   return(np.maximum(0, x))

#ReLU_output=[]
#for i in range(len(image_batch)):
#    ReLU_output += [tf.keras.activations.relu(image_batch[i], max_value= max_val)] #testing value